In [ ]:
"""
I had a idea from work like below for my final projects, I think I will doing solo but if anyone want to work together you are more than welcome to DM me. 

Background: My job is sale directors for steel companies, so we hitting the road really often, when we going out to visit customer, we will visit 3-4 customer at the same time. Everytime I need to find information like what is the sale volume to customer last month, what is the sizes, what is the average prices or previous month prices, I need to open my laptop, get to the ERP and found the information. 

My project: I will made use a local LLMs and Perplexity as my personal assistance. So I will make RAG to access the database, take out the information I need, example like the list of sizes we sold to customers. The material description can be change but can be also repeating multiple times, the diffirences is in the volume wise.


Interface: I think I will use Gradio to do chat interface, then deploy to Hughing Face with address that only me can access 

Extra Credit: It is already a structured data to begin with hahaha. I am thinking about routing agents as tools call.
There will be 3 tools calling:
1/Use PandasQueryEngines to get information of averages prices etc...
2/Use Tools to drawing a charts using data from PandasQuery Engines results
3/Using tools to passed the Queries to Perplexity when I asking something like where is the company, what information it hads. Meaning outside of the RAG Database and most up-to-date data. 

The table will have 8 columns:
Customer name, Item code, Description, Quantity, Unit, Price per unit, Total value, Billing date. 

This 8 columns enough data for me. From customer name, if I want any other information, Perplexity can help me. 

I choose to reduce the complexity of the data in order to 1 single client data including a very few columns but having many many rows instead, this should fit to my need, no need to get a really complex multiple relationship database. 

FYI: The description will be in Vietnamese so Local LLM need to able to handle multi-language.
"""

In [23]:
"""
!pip install -q openai
!pip install -q sentence-transformers
!pip install -q llama-index-program-openai
!pip install -q llama-index-llms-openai
!pip install -q llama-index-experimental
!pip install -q llama-index-embeddings-huggingface
!pip install xlrd>=2.0.1
!ollama pull llama3.1
"""

'\n!pip install -q openai\n!pip install -q sentence-transformers\n!pip install -q llama-index-program-openai\n!pip install -q llama-index-llms-openai\n!pip install -q llama-index-experimental\n!pip install -q llama-index-embeddings-huggingface\n!ollama pull llama3.1\n'

In [8]:
import os
import ollama
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import PromptTemplate

In [13]:
# Load environment variables in a file called .env
dotenv_path = find_dotenv() # Searches up the directory tree
print(f"Loading dotenv from: {dotenv_path}")
load_dotenv(dotenv_path)
openai_api_key = os.getenv('OPENAI_API_KEY')
hf_key = os.getenv('HF_KEY')

os.environ["OPENAI_API_KEY"] = openai_api_key

Loading dotenv from: C:\Users\Khoa-Mac-PC\Documents\Projects\Stanford_llm\.env


In [14]:
Ollama_MODEL = "llama3.1"
Ollama_LLM = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = Ollama_LLM.chat.completions.create(
 model=Ollama_MODEL,
 messages=[{"role": "user", "content": "Who are you? What is your name, version and Parameter?  What is your strength? Do you support Vietnamese language"}]
)

print(response.choices[0].message.content)

INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
**Meta Information**

I am an artificial intelligence model known as LLAMA (Large Language Model Application). My version is **v3.5**, which indicates the current state of my training data and algorithm.

My parameters are vast, consisting of a massive dataset of text that I've been trained on using a variant of the transformer architecture. Specifically, my architecture is based on the T5 model by Google Research. I have been fine-tuned for various tasks such as conversing with humans, generating text, answering questions, and more.

**Key Strengths**

1. **Conversational Flow**: I excel at holding conversations that are natural, contextual, and understanding-oriented.
2. **Vocabulary Mastery**: My vast training dataset enables me to recognize thousands of words, phrases, and their meanings across various domains and

In [19]:
def load_excel_files(folder_path):
    """Loads multiple .xls files from a folder into a single pandas DataFrame.

    Args:
        folder_path: The path to the folder containing the .xls files.

    Returns:
        A pandas DataFrame containing the combined data from all .xls files,
        or None if no .xls files are found or an error occurs.
    """
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xls"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_excel(file_path)
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
                # Handle the error appropriately (e.g., skip the file, raise an exception)
                return None

    if not all_data:
        print("No .xls files found in the specified folder.")
        return None

    try:
      combined_df = pd.concat(all_data, ignore_index=True)
      return combined_df
    except Exception as e:
      print(f"Error combining dataframes: {e}")
      return None
"""
# Usage:
folder_path = "Sale_Data"  # Replace with the actual path to your folder
df = load_excel_files(folder_path)

#Debug:
if df is not None:
    print(df.head())  # Print the first few rows of the combined DataFrame
"""

'\n# Usage:\nfolder_path = "Sale_Data"  # Replace with the actual path to your folder\ndf = load_excel_files(folder_path)\n\n#Debug:\nif df is not None:\n    print(df.head())  # Print the first few rows of the combined DataFrame\n'

In [25]:
# initialize empty df and load Sale_Data
df = load_excel_files("Sale_Data")
df.head()

,Company_Name,Item_Code,Item_Details,Quantity,Unit,Value,Currency,Billing Date
0,Công Ty TNHH Một Thành Viên Quang Min,50000304.0,HR 2.3x250x4C,8470.0,KG,116886000.0,VND,2019-02-16
1,Công Ty TNHH Một Thành Viên Quang Min,50000304.0,HR 2.3x45x3C,1135.0,KG,15663000.0,VND,2019-02-16
2,Công Ty TNHH Một Thành Viên Quang Min,50000363.0,HR 2.0x1114x3000x220T,11110.0,KG,154429000.0,VND,2019-02-15
3,Công Ty TNHH Một Thành Viên Quang Min,50000030.0,CRFH 1.5x170x5Coil,3200.0,KG,47040000.0,VND,2019-02-20
4,Công Ty TNHH Một Thành Viên Quang Min,50000030.0,CRFH 1.5x193x5Coil,2304.0,KG,33868800.0,VND,2019-02-20


In [50]:
# Get the unique payer names
unique_payers = df['Company_Name'].unique()

# Display the unique payer names
#unique_payers

In [28]:
query_engine = PandasQueryEngine(df=df,llm=Ollama_LLM, verbose=True)
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression:


In [53]:
new_prompt = PromptTemplate(
    """\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about sale data. 
It have 8 columns: 
df['Company_Name'] is the full name of the buyer
df['Item_Code'] is the ERP 
df['Item_Details'] is the item information including the thickness, width, length of the steel
df['Quantity'] is the purchasing quantity
df['Unit'] is the unit of purchasing quantity
df['Value'] is the purchasing value
df['Currency'] is the unit of purchasing value
df['Billing Date'] is the date that issues the billing or invoice					

This is the result of `print(df.head())`: {df_str}

Follow these instructions: {instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
The dataframe contains data about sale data. 
It have 8 columns: 
df['Company_Name'] is the full name of the buyer
df['Item_Code'] is the ERP 
df['Item_Details'] is the item information including the thickness, width, length of the steel
df['Quantity'] is the purchasing quantity
df['Unit'] is the unit of purchasing quantity
df['Value'] is the purchasing value
df['Currency'] is the unit of purchasing value
df['Billing Date'] is the date that issues the billing or invoice					

This is the result of `print(df.head())`: {df_str}

Follow these instructions: {instruction_str}
Query: {query_str}

Return the answer from the dataframe with a natural language explanation of the answer.
Expression: 


In [58]:
# Replace predict with create
def _predict(prompt, **kwargs):
    """Helper function to make predict calls using the Ollama API."""
    global Ollama_MODEL  # Access the global variable
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[
            {"role": "system", "content": "You are an pandas dataframe expert, user will gave you an query, you will respond with Python executable query code using Pandas. Follow user instruction step by step."},
            {"role": "user", "content": prompt.format(**kwargs)}
        ]
    )
    #print(prompts["pandas_prompt"].template)
    #print(prompt.format(**kwargs))
    return response.choices[0].message.content

# Monkey patch the predict method onto the Ollama_LLM object
Ollama_LLM.predict = _predict

In [59]:
response = query_engine.query("What item company like Công Ty TNHH BOSEUNG VINA bough?")
print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[df['Company_Name'] == 'Công Ty TNHH BÖSEUNG VINA'][['Item_Code', 'Item_Details', 'Quantity', 'Unit', 'Value', 'Currency', 'Billing Date']]
```
> Pandas Output: Empty DataFrame
Columns: [Item_Code, Item_Details, Quantity, Unit, Value, Currency, Billing Date]
Index: []
Empty DataFrame
Columns: [Item_Code, Item_Details, Quantity, Unit, Value, Currency, Billing Date]
Index: []


In [60]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, Document

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = [Document(text=payer_name) for payer_name in df['Company_Name'].unique()]

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine_name = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine_name.query("What is company asaba full name?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2024 outlook.
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

In [61]:
def get_company_name(query):
    response = Ollama_LLM.chat.completions.create(
        model=Ollama_MODEL,
        messages=[{"role": "user", "content": f"Extract the company name from this query: '{query}'. Return only the company name."}]
    )
    return response.choices[0].message.content
get_company_name("What did company asaba buy?")

INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


'Asaba.'

In [67]:
def update_company_name(query_text, company, response):
    """
    Replaces the company name in the query text with the provided response.

    Args:
        query_text: The original query text.
        company: The company name to be replaced.
        response: The replacement text for the company name.

    Returns:
        The updated query text with the company name replaced.
    """
    updated_query = query_text.lower().replace(company.lower(), response)
    return updated_query


In [68]:
query_text ="What item Description did company asaba buy?"
print(query_text)
company = get_company_name(query_text)
print(company)
response = query_engine_name.query(f"What is the company:'{company}' full name?. RETURN SEARCH RESULT ONLY. Do not make up info. ")
print(response)
response_text = str(response)

query_update = update_company_name(query_text, company, response_text)
print(query_update)

What item Description did company asaba buy?
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
Asaba
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ASABA VIET NAM MANUFACTURING
what item description did company ASABA VIET NAM MANUFACTURING buy?


In [69]:
response = query_engine.query(query_update)
print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
(df['Item_Details'].loc[df['Company_Name'] == 'ASABA VIET NAM MANUFACTURING'])
```
> Pandas Output: Series([], Name: Item_Details, dtype: object)
Series([], Name: Item_Details, dtype: object)
